In [1]:
# Import python modules
import os
import shutil
import requests
import pandas as pd
import datetime as dt
from PIL import Image
import plotly.graph_objects as go
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk

In [2]:
# Financial Modeling Prep API Key
def SEC(company):
    api = "425954bae24e265b01ea67e14c8c62b2"
    years = 4
    
    www = 'https://financialmodelingprep.com/api/v3'
    is_www = '/income-statement/'
    bs_www = '/balance-sheet-statement/'
    cf_www = '/cash-flow-statement/'
    r_www = '/ratios/'
    km_www = '/key-metrics/'
    p_www = '/profile/'
    rtg_www = '/rating/'
    es_www = '/earnings-surprises/'
    # sp_www = "/stock_peers"
    
    # Data to Import
    IS = requests.get(f'{www}{is_www}{company}?apikey={api}').json()
    BS = requests.get(f'{www}{bs_www}{company}?apikey={api}').json()
    CF = requests.get(f'{www}{cf_www}{company}?apikey={api}').json()
    Ratios = requests.get(f'{www}{r_www}{company}?apikey={api}').json()
    Metrics = requests.get(f'{www}{km_www}{company}?apikey={api}').json()
    Profile = requests.get(f'{www}{p_www}{company}?apikey={api}').json()
    Rating = requests.get(f'{www}{rtg_www}{company}?apikey={api}').json()
    Surprises = requests.get(f'{www}{es_www}{company}?apikey={api}').json()
    
    is_df = pd.DataFrame(IS)
    bs_df = pd.DataFrame(BS)
    cf_df = pd.DataFrame(CF)
    ratios_df = pd.DataFrame(Ratios)
    metrics_df = pd.DataFrame(Metrics)
    profile_df = pd.DataFrame(Profile)
    rating_df = pd.DataFrame(Rating)
    surprises_df = pd.DataFrame(Surprises)
    
    bs_df.sort_index(ascending=False, inplace=True)
    is_df.sort_index(ascending=False, inplace=True)
    cf_df.sort_index(ascending=False, inplace=True)
    bs_df = bs_df.iloc[-years:]
    is_df = is_df.iloc[-years:]
    cf_df = cf_df.iloc[-years:]

    # bs_df.to_csv('bs.csv', index=False)
    # is_df.to_csv('is.csv', index=False)
    # cf_df.to_csv('cf.csv', index=False)
    # ratios_df.to_csv('ratios.csv', index=False)
    # metrics_df.to_csv('metrics.csv', index=False)
    # profile_df.to_csv('profile.csv', index=False)
    # rating_df.to_csv('ratings.csv', index=False)
    # surprises_df.to_csv('surprises.csv', index=False)
    
    # Store all dataframes in a list
    dataframes = [is_df, bs_df, cf_df, ratios_df, metrics_df, profile_df, rating_df, surprises_df]
    return dataframes

In [10]:
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
from openai import OpenAI

# Initialize LLM client
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

# Store all dataframes in a list
dataframes = SEC('BCC')

def process_dataframe(df):
    """Send LLM request for a single dataframe."""
    df_summary = df.to_string(index=False)
    prompt_context = f"The following table shows financial data:\n\n{df_summary}\n\n"
    prompt_instruction = (
        "Please analyze the data like a true Wall Street broker and suggest to me what to do as investor."
    )
    final_prompt = prompt_context + prompt_instruction

    # Define chat history for the LLM
    history = [
        {"role": "system", "content": "You are an intelligent assistant. You always provide well-reasoned answers."},
        {"role": "user", "content": final_prompt},
    ]

    # Make the LLM request
    completion = client.chat.completions.create(
        model="QuantFactory/Llama-3-8B-Instruct-Finance-RAG-GGUF",
        messages=history,
        temperature=0.7,
    )

    # Extract and return the response
    return completion.choices[0].message.content

# Process all dataframes in parallel
results = []
with ThreadPoolExecutor() as executor:
    futures = {executor.submit(process_dataframe, df): df for df in dataframes}
    for future in futures:
        try:
            result = future.result()
            results.append(result)
        except Exception as e:
            print(f"Error processing dataframe: {e}")

# Combine all individual analyses
combined_analysis = "\n\n".join(results)

# Request a combined conclusion from the LLM
final_prompt = (
    f"The following are analyses of financial data from multiple tables:\n\n{combined_analysis}\n\n"
    "Based on these analyses, provide an overall conclusion about the company's financial performance and give an overall sentiment about this company's financial health."
)

history = [
    {"role": "system", "content": "You are an intelligent assistant. You always provide well-reasoned answers."},
    {"role": "user", "content": final_prompt},
]

completion = client.chat.completions.create(
    model="QuantFactory/Llama-3-8B-Instruct-Finance-RAG-GGUF",
    messages=history,
    temperature=0.7,
)

# Extract the combined conclusion
combined_conclusion = completion.choices[0].message.content

# Output final result
print("Individual Analyses:")
print(combined_analysis)
print("\nOverall Conclusion:")
print(combined_conclusion)


Individual Analyses:
Based on this financial data, I would recommend holding onto BCC stock for the long-term. The company has consistently demonstrated strong revenue growth, with a gross profit ratio that has been trending upward over the past three years. Additionally, operating expenses have remained relatively stable, which indicates a potential for increased profitability.

The recent decline in EPS and net income may be attributed to various factors including market conditions and changes in interest rates. However, given the company's history of growth and strong financial position, I believe that this is an attractive opportunity to invest in BCC stock at current levels.

It is essential to keep in mind that past performance does not necessarily guarantee future success, and it is always important to conduct thorough research before making investment decisions. As a long-term investor, I would recommend holding onto the stock for a period of time to allow the company's financi